# Analytische Vaardigheden




Importeren van de dependencies


In [37]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import datetime
pd.options.mode.chained_assignment = None

## Het inladen van de dataset
Eerst laden we de dataset in

In [38]:
indexes = pd.read_csv('GSPC.csv')
indexes.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


## Het schoonmaken
Dan gaan we de dataset schoonmaken en alle kolommen droppen die we niet meer gaan gebruiken. Ook maken we van de kolomnamen kleine letters want dat is handig!

In [39]:
indexes['date'] = pd.to_datetime(indexes['Date'])
indexes['open'] = indexes['Open']
columns_to_keep = [
    'date',
    'open'
]
indexes = indexes[columns_to_keep]
indexes.head()

,date,open
0,1950-01-03,16.66
1,1950-01-04,16.85
2,1950-01-05,16.93
3,1950-01-06,16.98
4,1950-01-09,17.08


Nu gaan we de startjaar en eindjaar bepalen

In [40]:
startjaar = 2012
duur = 4
eindjaar = startjaar + duur
eindjaar

2016

Stel de rente in op de lening en het jaarlijks rendement op de staatslening beleggingen

In [41]:
leningrente = 0.0234
bondsrendement = 0.0171

Nu gaan we een nieuwe Dataframe maken waar al onze resultaten in komen, we maken er eerst eentje met alle datums waar we gaan investeren. 

In [42]:
jaren = pd.date_range(str(startjaar) + "-09-01", periods=duur + 1, freq='AS-SEP')
df = pd.DataFrame({'date': jaren, 'schuld met rente 0.00': 0.0, 'schuld met rente {}'.format(leningrente*100): 0.0})
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34
0,2012-09-01,0.0,0.0
1,2013-09-01,0.0,0.0
2,2014-09-01,0.0,0.0
3,2015-09-01,0.0,0.0
4,2016-09-01,0.0,0.0


Nu gaan we de maandelijkse studiefinanciering bepalen, deze is te vinden op de site van DUO.

In [43]:
lening = 486.08
beurs = 396.39
collegegeldkrediet = 173.59
totaal = lening + beurs + collegegeldkrediet
totaal

1056.06

Dat is wat we per maand kunnen krijgen van Ome duo. Per jaar is dat dus:

In [44]:
totaal = totaal * 12
totaal

12672.72

Op 1 september van het eerste jaar hebben we nog geen geld, want dan beginnen we pas, dus dan is de schuld 0

In [45]:
df['schuld met rente 0.00'][0] = 0
df['schuld met rente {}'.format(leningrente*100)][0] = 0
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34
0,2012-09-01,0.0,0.0
1,2013-09-01,0.0,0.0
2,2014-09-01,0.0,0.0
3,2015-09-01,0.0,0.0
4,2016-09-01,0.0,0.0


Doordat er in de dataset niet altijd een 1 september te vinden is, omdat er in het weekend de markten niet open zijn, gaan we kijken wat de eerstvolgende datum is dat er wel een waarde beschikbaar is

In [46]:
def get_day_value(date):
    global df, indexes
    if len(indexes[indexes['date'] != date]) != len(indexes) - 1:
        return get_day_value(date + datetime.timedelta(days=1))
    else:
        return indexes[indexes['date'] == date]

for index, row in df.iterrows():
    df['date'][index] = get_day_value(row['date']).iloc[0]['date']
df.head()
df = pd.merge(df, indexes, on='date')
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34,open
0,2012-09-04,0.0,0.0,1406.540039
1,2013-09-03,0.0,0.0,1635.949951
2,2014-09-02,0.0,0.0,2004.069946
3,2015-09-01,0.0,0.0,1970.089966
4,2016-09-01,0.0,0.0,2171.330078


We voegen nog de kolommen toe voor de aantal aandelen en voor de rest van het geld

In [47]:
df['aandelen'] = 0
df['rest'] = 0
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34,open,aandelen,rest
0,2012-09-04,0.0,0.0,1406.540039,0,0
1,2013-09-03,0.0,0.0,1635.949951,0,0
2,2014-09-02,0.0,0.0,2004.069946,0,0
3,2015-09-01,0.0,0.0,1970.089966,0,0
4,2016-09-01,0.0,0.0,2171.330078,0,0


Nu komt het gedeelte waar we elk jaar berekenen 

In [48]:
for index, row in df.iterrows():
    if index == 0:
        # Jaar 1 totaal = schuld
        df.loc[index, 'schuld met rente 0.00'] = totaal
        df.loc[index, 'schuld met rente {}'.format(leningrente*100)] = totaal
        df.loc[index, 'aandelen'] = np.floor(totaal / row['open'])
        df.loc[index, 'rest'] = totaal - df.loc[index, 'aandelen'] * row['open']
        # Waarde staatsleningen jaar 1 = totaal, er is immers nog niks belegd
        df.loc[index, 'waarde staatsleningen'] = totaal
    elif index != len(df) - 1:
        # Neem rente mee in de schuld = totaal * (1 + rente)^aantal jaren
        df.loc[index, 'schuld met rente 0.00'] = df.iloc[index - 1]['schuld met rente 0.00'] + totaal
        df.loc[index, 'schuld met rente {}'.format(leningrente*100)] = round(df.iloc[index - 1]['schuld met rente {}'.format(leningrente*100)] + (totaal * (1 + leningrente)**index),2)
        df.loc[index, 'aandelen'] = df.loc[index - 1, 'aandelen'] + np.floor((totaal + df.loc[index - 1, 'rest']) / row['open'])
        df.loc[index, 'rest'] = (totaal + df.loc[index -1, 'rest']) - np.floor((totaal + df.loc[index - 1, 'rest']) / row['open']) * row['open']
        df.loc[index, 'waarde staatsleningen'] = round(df.loc[index -1, 'waarde staatsleningen'] + (totaal * (1 + bondsrendement)**index),2)
    else:
        # Neem rente mee in eindstand, voeg rente toe op de vorige schuld
        df.loc[index, 'schuld met rente 0.00'] = round(df.loc[index - 1, 'schuld met rente 0.00'])
        df.loc[index, 'schuld met rente {}'.format(leningrente*100)] = round(df.loc[index - 1, 'schuld met rente {}'.format(leningrente*100)] * (1 + leningrente ), 2)
        df.loc[index, 'aandelen'] = df.loc[index - 1, 'aandelen']
        df.loc[index, 'rest'] = df.loc[index - 1, 'rest']
        # Neem de laatste waarde en voeg daar nog eenmaal de rente op toe
        df.loc[index, 'waarde staatsleningen'] = round(df.loc[index - 1, 'waarde staatsleningen'] * (1 + bondsrendement),2)
        
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34,open,aandelen,rest,waarde staatsleningen
0,2012-09-04,12672.72,12672.72,1406.540039,9.0,13.859649,12672.72
1,2013-09-03,25345.44,25641.98,1635.949951,16.0,1234.929992,25562.14
2,2014-09-02,38018.16,38914.72,2004.069946,22.0,1883.230316,38671.97
3,2015-09-01,50690.88,52498.04,1970.089966,29.0,765.320554,52005.98
4,2016-09-01,50691.00,53726.49,2171.330078,29.0,765.320554,52895.28


Eind portofolio waarde SP500:

In [49]:
round(df.loc[len(df) - 1, 'aandelen'] * df.loc[len(df) - 1, 'open'] + df.loc[len(df) - 1, 'rest'], 2)

63733.89

Totaal rendement bij SP500 met en zonder rente

In [50]:
# Aantal aandelen * open waarde uitstap jaar 
sp500zonderrente = round(df.loc[len(df) - 1, 'aandelen'] * df.loc[len(df) - 1, 'open'] + df.loc[len(df) - 1, 'rest'] - df.loc[len(df) -1, 'schuld met rente 0.00'], 2)
sp500metrente = round(df.loc[len(df) - 1, 'aandelen'] * df.loc[len(df) - 1, 'open'] + df.loc[len(df) - 1, 'rest'] - df.loc[len(df) -1, 'schuld met rente {}'.format(leningrente*100)], 2)

# Waarde staatsleningen - schuld met rente x.xx
staatsleningenzonderrente = round( df.loc[len(df) -1, 'waarde staatsleningen'] - df.loc[len(df) -1, 'schuld met rente 0.00'], 2)
staatsleningenmetrente = round(df.loc[len(df) -1, 'waarde staatsleningen'] - df.loc[len(df) -1, 'schuld met rente {}'.format(leningrente*100)], 2)


# Print de eindstand
print('Rendement in scenario {} tot en met {} met een jaarlijkse inleg van {:.2f} euro.'.format(startjaar, eindjaar, totaal))
print('SP500 rendement zonder rente: {:.2f} euro'.format(sp500zonderrente))
print('SP500 rendement met rente: {:.2f} euro'.format(sp500metrente))
print('Bonds rendement zonder rente: {:.2f} euro'.format(staatsleningenzonderrente))
print('Bonds rendement met rente: {:.2f} euro'.format(staatsleningenmetrente))

Rendement in scenario 2012 tot en met 2016 met een jaarlijkse inleg van 12672.72 euro.
SP500 rendement zonder rente: 13042.89 euro
SP500 rendement met rente: 10007.40 euro
Bonds rendement zonder rente: 2204.28 euro
Bonds rendement met rente: -831.21 euro
